# SVM Classifier

## Librairies

In [1]:
from pandas import DataFrame
import os
from skimage.transform import resize
from skimage.io import imread
from sklearn import svm, metrics, datasets
from sklearn.model_selection import GridSearchCV, train_test_split
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

## Show categories in a dataset

In [2]:
container_path = "Datasets/Mint"
image_dir = Path(container_path)
folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
Categories = [fo.name for fo in folders]

In [3]:
Categories

['Anthracnose', 'healthy', 'Powdery-mildew', 'Rust', 'Verticillium-wilt']

## Image Loader

In [4]:
def load_image_files(container_path):
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    Categories = [fo.name for fo in folders]

    flat_data_arr=[] #input array
    target_arr=[] #output array
    datadir= container_path
    img_loaded = 0

    for i in Categories:
        
        print(f'loading... category : {i}')
        path=os.path.join(datadir,i)
        for img in os.listdir(path):
            try:
                img_array=imread(os.path.join(path,img))
                img_resized=resize(img_array,(150,150,3))
                flat_data_arr.append(img_resized.flatten())
                target_arr.append(Categories.index(i))
                img_loaded += 1
            except:
                pass
        print(f'loaded category:{i} successfully')
        
    flat_data=np.array(flat_data_arr)
    target=np.array(target_arr)
    df=DataFrame(flat_data) #dataframe
    df['Target']=target
    X=df.iloc[:,:-1] #input data 
    y=df.iloc[:,-1] #output data
    
    return X,y,img_loaded

In [5]:
X,y,img_loaded = load_image_files("Datasets/Mint")
print("Number of images loaded: ",img_loaded)

loading... category : Anthracnose
loaded category:Anthracnose successfully
loading... category : healthy
loaded category:healthy successfully
loading... category : Powdery-mildew


## Process dataset

In [23]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=109)
print('Splitted Successfully')

Splitted Successfully


In [24]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
svc = svm.SVC()
model = GridSearchCV(svc, param_grid)

In [25]:
model.fit(X_train,y_train)
print('The Model is trained well with the given images')

## Prediction on Test

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
print("The predicted Data is :")
print(y_pred)
print("The actual data is:")
print(np.array(y_test))

In [ ]:
from sklearn.metrics import accuracy_score
print(f"The model is {accuracy_score(y_pred,y_test)*100}% accurate")

In [ ]:
print("Classification report for - \n{}:\n{}\n".format(
    model, metrics.classification_report(y_test, y_pred)))

## Prediction on single image

In [ ]:
imgPath = r"Datasets\images_test\dollar_bill\image_0002.jpg" 

img=imread(imgPath)
plt.imshow(img)
plt.show()
img_resize=resize(img,(150,150,3))
imgFlat=[img_resize.flatten()]

In [ ]:
pred=model.predict(imgFlat)


In [ ]:
indicePred = pred[0]
categoriePred = Categories[indicePred]
print("The predicted image is : "+categoriePred)

## Save Model

In [ ]:
# save the model to disk
import pickle
filename = 'svm_model.sav'
pickle.dump(model, open(filename, 'wb'))

## Load a model


In [ ]:
# # load the model from disk
import pickle
filename = 'svm_model.sav'
model = pickle.load(open(filename, 'rb'))
result = model.score(X_test, y_test)
print(result)